## What are graphs?

Tensorflow 2.0 runs in two modes:
* **eagrly:** normal Python execution
* **graphs:** portable (no Python interpreter needed), offer speed improvements

graphs are made of *`tf.Operation`* and *`tf.Tensor`*

## How is a graph created?
using `@tf.function` or calling `tf.function()`

a function is transformed into a PolymorphicFunction

* `tf.function` is used to make graphs out of programs, for better performance and deployability
* `tf.function` creates Python-indpendent dataflow graphs
* `tf.function` works best with tf ops and NumPy, might have problems with Python native commands

In [6]:
import tensorflow as tf

In [2]:
# Define a Python function.
def a_regular_function(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x

In [3]:
# Python function --> PolymorphicFunction (function that uses grapsh)
# The Python type of `a_function_that_uses_a_graph` will now be a
# `PolymorphicFunction`.
a_function_that_uses_a_graph = tf.function(a_regular_function)

In [7]:
# try execution w/ and w/out graph
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

orig_value = a_regular_function(x1, y1, b1).numpy()
tf_function_value = a_function_that_uses_a_graph(x1, y1, b1).numpy()
orig_value, tf_function_value

(array([[12.]], dtype=float32), array([[12.]], dtype=float32))

**Note:** `tf.function` applies to all the functions called inside it too.

In [9]:
def inner_function(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x

# Using the `tf.function` decorator makes `outer_function` into a
# `PolymorphicFunction`.
@tf.function
def outer_function(x):
  y = tf.constant([[2.0], [3.0]])
  b = tf.constant(4.0)

  return inner_function(x, y, b)

# Note that the callable will create a graph that
# includes `inner_function` as well as `outer_function`.
outer_function(tf.constant([[1.0, 2.0]])).numpy()

array([[12.]], dtype=float32)

## But how is the graph really created?!

* tf operations are esily trasformed
* Python code is converted to graph-generating code throug the **Autograph library (`tf.autograph`)**

Here's a graph example of the Relu function
* use `tf.autograph.to_code(func)` to view the function
* use `.get_concrete_function(<func input>).graph.as_graph_def()` to view the graph itself

In [10]:
def simple_relu(x):
    if tf.greater(x, 0):
        return x
    else:
        return 0

# to `PolymorphicFunction`
tf_simple_relu = tf.function(simple_relu)

# This is the graph-generating output of AutoGraph.
print(tf.autograph.to_code(simple_relu))

def tf__simple_relu(x):
    with ag__.FunctionScope('simple_relu', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal do_return, retval_
            (do_return, retval_) = vars_

        def if_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = ag__.ld(x)
            except:
                do_return = False
                raise

        def else_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = 0
            except:
                do_return = False
                raise
        ag__.if_stmt(ag__.converted_call(ag__.ld(tf).greater, (ag__.ld(x), 0), None, fscope), if_bo

In [11]:
print(tf_simple_relu.get_concrete_function(tf.constant(1)).graph.as_graph_def())

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
}
node {
  name: "Greater/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 0
      }
    }
  }
}
node {
  name: "Greater"
  op: "Greater"
  input: "x"
  input: "Greater/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "cond"
  op: "StatelessIf"
  input: "Greater"
  input: "x"
  attr {
    key: "Tcond"
    value {
      type: DT_BOOL
    }
  }
  attr {
    key: "Tin"
    value {
      list {
        type: DT_INT32
      }
    }
  }
  attr {
    key: "Tout"
    value {
      list {
        type: DT_BOOL
        type: DT_INT32
      }
    }
  